In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas

In [3]:
df = pd.read_csv("data/open_tasks_Corn_EKV_DE.csv")
df.dropna(subset=["description"], inplace=True)
print(len(df))
df.head()

130


,documentId,tenant,language,title,description,context,topic,hints,solutionSteps,replacements,resource,sampleSolutions,stepifier,word_count
14989,81xdUMIFiyC7gTSwml1MTv,Cornelsen DE,DE,Archive | 4.1 Erlebnislyrik | Kennzeichen,Was ist ein Kennzeichen der Erlebnislyrik? Ant...,NaN,"{'id': 5350, 'name': 'Essay 3', 'description':...",[],[],NaN,NaN,[''],KEYWORD,11
14990,7fE9sk0x2amalma0takBp5,Cornelsen DE,DE,Archive | 3.3 Schlussteil Interpretationsaufsatz,Welches Vorgehen planen Sie für den Schlusstei...,NaN,"{'id': 5349, 'name': 'Essay 1', 'description':...",[],[],NaN,NaN,[''],KEYWORD,14
14991,7CUm7BcYBG79uI8VPMUKCo,Cornelsen DE,DE,Archive | 3.2 Gedicht | Inhaltliche Aspekte,Auf welchen inhaltlichen Aspekt hin würden Sie...,NaN,"{'id': 5349, 'name': 'Essay 1', 'description':...",[],[],NaN,NaN,[''],KEYWORD,15
15031,6eVh0sUk73laFRNimbk7MC,Cornelsen DE,DE,DaF | Relativpronomen,Mache aus den zwei Sätzen einen Relativsatz.Ic...,NaN,"{'id': 5357, 'name': 'DaF', 'description': '',...",[],"['Ich arbeite gern mit dem Laptop, den ich let...",NaN,NaN,[''],KEYWORD,19
15032,avYO5lPPpNR7MVAKdARpVK,Cornelsen DE,DE,DaF | Plusquamperfekt,Was war zuerst? Schreibe einen Satz mit nachde...,NaN,"{'id': 5357, 'name': 'DaF', 'description': '',...",[],['Nachdem ich eine neue Wohnung gefunden hatte...,NaN,NaN,[''],KEYWORD,19


In [4]:
# Back Translation Augmenter
text = df.loc[14989]['description']

back_translation_aug = naw.BackTranslationAug(
    from_model_name='facebook/wmt19-de-en', 
    to_model_name='facebook/wmt19-en-de'
)
augmented_text = back_translation_aug.augment(text)[0]
print(text)
print(augmented_text)

Some weights of FSMTForConditionalGeneration were not initialized from the model checkpoint at facebook/wmt19-de-en and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of FSMTForConditionalGeneration were not initialized from the model checkpoint at facebook/wmt19-en-de and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Was ist ein Kennzeichen der Erlebnislyrik? Antworten Sie in einem Satz.
Was ist ein Charakteristikum von Erfahrungsdichtung? Antwort in einem Satz.


In [5]:
df_aug = pd.DataFrame()
df_aug ['description'] = df['description'].apply(lambda text: back_translation_aug.augment(text)[0])

In [12]:
df_all = pd.concat([df_aug, df[['description']]]).reset_index(drop=True)
df_all

,description
0,Was ist ein Charakteristikum von Erfahrungsdic...
1,Was planen Sie für den letzten Teil des Interp...
2,Wie würden Sie das Gedicht interpretieren? Sch...
3,Ich arbeite gerne mit dem Laptop. Den Laptop h...
4,"Was war zuerst? Schreiben Sie einen Satz, nach..."
...,...
255,Bei einer totalen Mondfinsternis kann man stat...
256,"Begründe, warum eine Treppe auch eine schiefe ..."
257,In diesen Texten kommen drei unterschiedliche ...
258,Ein Gegenstand aus Eisen wird sowohl vom Nord-...


In [13]:
df_all.to_csv("data/augmented_open_tasks_Corn_EKV.csv", index_label=False)

<hr>

https://github.com/makcedward/nlpaug/blob/master/example/textual_augmenter.ipynb